# Importaciones

In [ ]:
from imports import *
from trading_environment import *
from strategies import *

NameError: name 'Tuple' is not defined

# Strategies

In [ ]:
# 1) Definimos dos grids de parámetros:
dca_grid = {
    'amount': [500, 1000, 2000],
    'freq':   ['W', 'M', 'D']
}

ma_grid = {
    'long_window':  [50, 100, 200],
    'short_window': [5, 20, 50]
}

# 2) Creamos dos colecciones, una para cada tipo de estrategia
#    -> no cambia: __init__ sigue aceptando (strategy_cls, param_grid)
dca_collection = StrategyCollection(DcaStrategy, dca_grid)
ma_collection  = StrategyCollection(MovingAverageCrossStrategy, ma_grid)

print(f"DCA strategies: {dca_collection.names()}")
print(f"MA  strategies: {ma_collection.names()}")

# 3) Ejecutamos backtests de todas las estrategias
symbol = 'BTC-USD'
common_kwargs = {
    'period':           '2y',
    'interval':         '1d',
    'initial_capital':  5000.0,
    'transaction_cost': 0.001,
}

# Backtest DCA (sin stop loss ni RiskManager)
equity_dca = dca_collection.backtest_all(
    symbol=symbol,
    **common_kwargs
)

# Backtest MA (con percent risk manager y stop-loss a 20000)
rm = RiskManager(method='pct', risk_pct=0.02)
equity_ma = ma_collection.backtest_all(
    symbol=symbol,
    **common_kwargs,
    risk_manager=rm,
    stop_loss=20000
)

# 4) Guardamos todas las estrategias en disco
strat_mgr = StrategyManager(directory=STRAT_DIR)
dca_collection.save_all(strat_mgr)
ma_collection.save_all(strat_mgr)

# 5) —> AHORA: Usamos el nuevo ReportManager.summarize() en lugar de compare_performance()
#    Ya no necesitamos recolectar y print/save por separado, todo lo maneja summarize()

# Recolectamos analyzers (igual que antes)
analyzers = {}
for name, eq in equity_dca.items():
    strat = dca_collection.strategies[name]
    analyzers[name] = PerformanceAnalyzer(eq, strat.returns)

for name, eq in equity_ma.items():
    strat = ma_collection.strategies[name]
    analyzers[name] = PerformanceAnalyzer(eq, strat.returns)

# Generamos y guardamos un informe comparativo
rep_mgr = ReportManager(directory=REPORT_DIR)

# ----> aquí el cambio clave:
perf_df = rep_mgr.summarize(
    analyzers,           # puede ser dict de PerformanceAnalyzer
    save=True,           # guarda comparación a CSV
    do_print=True        # imprime por consola
)

# Y luego podemos seguir usando los plots
rep_mgr.plot_metrics(perf_df)
rep_mgr.plot_equity_curves({**equity_dca, **equity_ma})


# FIN